In [5]:
import os
import requests
import gradio as gr
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from openai import OpenAI

In [3]:
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [6]:
gemma_model = ChatGroq(
    model_name = 'Gemma-7b-It',
    groq_api_key=groq_api_key,
    temperature=0.7,
    streaming=True
)


llama_model = ChatGroq(
    model_name='Llama-3.2-3b-Preview',
    groq_api_key=groq_api_key,
    temperature=0.7,
    streaming=True
)

openai = OpenAI()

In [7]:
system_message = "You are a helpful assistant"

In [8]:
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    return completion.choices[0].message.content

In [9]:
message_gpt("What is today's date?")

"Today's date is November 3, 2023."

## UI

In [11]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [14]:
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [15]:
system_message = "You are a helpful assistant that responds in markdown"

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    
    res = ""
    for chunk in stream:
        res += chunk.choices[0].delta.content or ""
        yield res

In [16]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


## Company Brochure

In [17]:
class Website:
    url: str
    title: str
    text: str
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
    def get_contents(self):
        return f"Webpage Title: \n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits."

In [19]:
def stream_brochure(company_name, url):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    result = stream_gpt(prompt)
    yield from result

In [20]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
